In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
import xgboost

# Readng Data

In [ ]:
df =pd.read_csv("../input/titanic/train.csv")
df

# Exploratory Data analysis

In [ ]:
sns.countplot(x="Sex",data=df)

In [ ]:
sns.countplot(x="Sex",hue="Survived",data=df)

In [ ]:
sns.distplot(x=df.Age,hist=True)

In [ ]:
sns.distplot(x=df.Fare,hist=True)

In [ ]:
sns.countplot(x="Pclass",hue="Survived",data=df)

In [ ]:
sns.countplot(x="Survived",hue="Pclass",data=df)

In [ ]:
sns.countplot(x="Embarked",data=df)

In [ ]:
scatter_matrix(df)
plt.show()

In [ ]:
print(df.corr())
sns.heatmap(df.corr())

# Data Processing

In [ ]:
df.isnull()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df["Age"].fillna(df["Age"].mean(),inplace=True)
df["Embarked"].fillna(df["Embarked"].mode()[0],inplace=True)

In [ ]:
df.info()

In [ ]:
data=df.drop(["PassengerId","Name","Ticket","Cabin"],axis=1)
data

In [ ]:
#from sklearn.preprocessing import OneHotEncoder
#ohe=OneHotEncoder()

In [ ]:
dum=pd.get_dummies(data,columns=["Pclass","Sex","Embarked"])

In [ ]:
#removing highly correlated variables 

In [ ]:
print(dum.corr())

In [ ]:
upd_dum=dum.drop(["Pclass_2","Sex_male","Embarked_Q"],axis=1)
#upd_dum["Fare"]=upd_dum["Fare"]/32
#upd_dum["Age"]=upd_dum["Age"]/30

In [ ]:
upd_dum.info()

In [ ]:
sns.heatmap(upd_dum.isnull(),cbar=False)

In [ ]:
upd_dum

In [ ]:
#upd_dum["scale_fare"]=upd_dum["Fare"]/20

In [ ]:
#dummy=upd_dum.drop(["Fare"],axis=1)

In [ ]:
X=upd_dum.drop(["Survived"],axis=1)
y=upd_dum["Survived"]

# Model Selection

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score(RandomForestClassifier(n_estimators=65),X,y,cv=3).mean()

In [ ]:
cross_val_score(LogisticRegression(),X,y,cv=3).mean()

In [ ]:
cross_val_score(GaussianNB(),X,y,cv=3).mean()

In [ ]:
cross_val_score(MultinomialNB(),X,y,cv=3).mean()

In [ ]:
#This is the bst model
cross_val_score(xgboost.XGBClassifier(n_estimators=20),X,y,cv=3).mean()

In [ ]:
model=xgboost.XGBClassifier(n_estimators=20)
model.fit(X,y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.8)
mod=xgboost.XGBClassifier(n_estimators=20)
mod.fit(X_train,y_train)
p=mod.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(p,y_test)

In [ ]:
m=RandomForestClassifier(n_estimators=65)
m.fit(X,y)

Used whole given data as training set

In [ ]:
#n_estimators=[int(x) for x in np.linspace(start=10,stop=110,num=10)]
#max_features=["auto","sqrt"]
#max_depth=[2,6]
#bootstrap=[True,False]

In [ ]:
#random_grid={"n_estimators": n_estimators, 
#            "max_features": max_features, 
#          "max_depth": max_depth,
#            "bootstrap": bootstrap }
#print(random_grid)

In [ ]:
#from sklearn.ensemble import RandomForestClassifier

In [ ]:
#model=RandomForestClassifier() 
#from sklearn.model_selection import GridSearchCV
#grid= GridSearchCV(estimator=model, param_grid=random_grid, cv=5,verbose=2,n_jobs=4)


In [ ]:
#grid.fit(X,y)

In [ ]:
#grid.best_params_

In [ ]:
#grid.score(X,y)

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test=train_test_split(X,y,train_size=0.85)
#X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
#model=RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1) 
#model.fit(X,y)

In [ ]:
#predict=grid.predict(X)
#predict

In [ ]:
#from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
#print(accuracy_score(predict,y))

In [ ]:
#print(confusion_matrix(y_test,predict))

In [ ]:
#print(classification_report(y_test,predict))

# Prepraing testing data

In [ ]:
test=pd.read_csv("../input/titanic/test.csv")
test

In [ ]:
tst=test.drop(["PassengerId","Name","Ticket","Cabin"],axis=1)
tst

In [ ]:
sns.heatmap(tst.isnull(),cbar=False)

In [ ]:
tst.info()

In [ ]:
#Replacing null values in test dataset with mean and mode
tst.Fare=tst.Fare.fillna(tst.Fare.mean())
tst.Age=tst.Age.fillna(tst.Age.mean())
tst.isnull().sum()

In [ ]:
dummy=pd.get_dummies(tst,columns=["Pclass","Sex","Embarked"])
upda_dum=dummy.drop(["Pclass_2","Sex_male","Embarked_Q"],axis=1)
#upda_dum["Fare"]=upda_dum["Fare"]/32
#upda_dum["Age"]=upda_dum["Age"]/30
upda_dum

In [ ]:
#dumy=upda_dum.drop(["Fare"],axis=1)

In [ ]:
results=m.predict(upda_dum)
result=pd.DataFrame(results,columns=["Survived"])
result

In [ ]:
sns.countplot(data=result,x="Survived")

In [ ]:
sub=pd.read_csv("../input/titanic/gender_submission.csv")
sub

In [ ]:
PassengerId=sub["PassengerId"]

# submission

In [ ]:
submission=pd.concat([PassengerId,result],axis=1)
submission

In [ ]:
submission.to_csv("submission.csv",index=False)

In [ ]:
!kaggle competitions submit -c titanic -f submission.csv -m "Message"